In [1]:
import polars as pl
import torch
from torchvision.transforms import v2
from PIL import Image
import io
import matplotlib.pyplot as plt

from pathlib import Path

DATA_PATH = Path.cwd().parent / "data"

In [2]:
transforms = v2.Compose([
    v2.ToImage(),  # Convert to tensor, only needed if you had a PIL image
    v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
    v2.Resize(size=(224, 224), antialias=True),  # Or Resize(antialias=True)
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
data = pl.read_parquet(DATA_PATH / "food101-train.parquet")

AttributeError: 'LazyFrame' object has no attribute 'with_column'